# Particle database

This notebook explains the difference between the particle list in `pycompwa`
and the list used in the ComPWA C++ back-end. The two are only related the
way they import and export XML files.

## pycompwa particle list

In [1]:
from pycompwa.expertsystem.ui.system_control import StateTransitionManager
from pycompwa.expertsystem.state.particle import particle_list

initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma"), ("pi0"), ("pi0")]
tbd_manager = StateTransitionManager(
    initial_state, final_state,
    formalism_type='helicity',
    topology_building='isobar')
original_number_of_entries = len(particle_list)
print('loaded', original_number_of_entries, 'particles')

loaded 70 particles


### (1) through Python

In [2]:
particle_list['D*(2010)+']

{'@Name': 'D*(2010)+',
 'Pid': '413',
 'Parameter': OrderedDict([('@Type', 'Mass'),
              ('@Name', 'Mass_D*(2010)+'),
              ('Value', '2.01'),
              ('Fix', 'true')]),
 'QuantumNumber': [OrderedDict([('@Class', 'Spin'),
               ('@Type', 'Spin'),
               ('@Value', '1')]),
  OrderedDict([('@Class', 'Int'), ('@Type', 'Charge'), ('@Value', '1')]),
  OrderedDict([('@Class', 'Int'), ('@Type', 'Parity'), ('@Value', '-1')]),
  OrderedDict([('@Class', 'Spin'),
               ('@Type', 'IsoSpin'),
               ('@Value', '0.5'),
               ('@Projection', '0.5')]),
  OrderedDict([('@Class', 'Int'), ('@Type', 'Charm'), ('@Value', '1')]),
  OrderedDict([('@Class', 'Int'),
               ('@Type', 'BaryonNumber'),
               ('@Value', '0')])],
 'DecayInfo': OrderedDict([('@Type', 'relativisticBreitWigner'),
              ('FormFactor', OrderedDict([('@Type', 'BlattWeisskopf')])),
              ('Parameter',
               [OrderedDict([('@Type', '

Although this is a bit arduous to work with, it is possible to directly
manipulate the entries in there through Python. Whatever modifications to
the ``particle_list`` instance are later on also used in the expert system.
In this case, for instance, we can set a wider width for the $D^*$:

In [3]:
particle_list['D*(2010)+']['DecayInfo']['Parameter'][0]['Value'] = 0.01

In [4]:
from pycompwa.expertsystem.state import particle

In [5]:
energies = [4180, 4220, 4420, 4600]  # MeV

for counter, energy in enumerate(energies, 1):
    new_particle = particle.get_particle_copy_by_name("EpEm")
    new_particle["Parameter"]["Value"] = energy / 1e3  # GeV
    pid = int(new_particle['Pid']) + counter
    new_particle["@Name"] += str(energy)  # new name to not overwrite the old
    new_particle["Pid"] = str(pid) # set unique PID
    particle.add_to_particle_list(new_particle)

assert original_number_of_entries + len(energies) == len(particle_list)

In [6]:
from IPython.display import display
display(particle.particle_list["EpEm"]["Parameter"])
display(particle.particle_list["EpEm4180"]["Parameter"])

OrderedDict([('@Type', 'Mass'),
             ('@Name', 'Mass_epem'),
             ('Value', '4.300'),
             ('Fix', 'true')])

OrderedDict([('@Type', 'Mass'),
             ('@Name', 'Mass_epem'),
             ('Value', 4.18),
             ('Fix', 'true')])

In [7]:
particle.particle_list['D*(2010)+']['DecayInfo']['Parameter'][0]['Value'] = 0.01

Finally, you can write particle to another XML file, so that you can have
this particle list manipulation in a separate Python script:

In [8]:
particle.write_particle_list_to_xml('new_particle_list.xml')

### (2) through XML

Alternatively, you can copy the default XML file and manipulate it by hand
(copy a section, tweak some names, change some parameters) and load it as
follows:

In [9]:
particle.load_particle_list_from_xml('../particle_list.xml')

Note that duplicate values in the particle list are overwritten, but that old
values are kept:

In [10]:
print('Number of entries in particle_list')
print('  old:', original_number_of_entries)
print('  new:', len(particle.particle_list))

Number of entries in particle_list
  old: 70
  new: 74


In [11]:
particle_list['D*(2010)+']['DecayInfo']['Parameter'][0]['Value']

'83.4E-6'

In [12]:
particle_list['EpEm4420']['Parameter']

OrderedDict([('@Type', 'Mass'),
             ('@Name', 'Mass_epem'),
             ('Value', 4.42),
             ('Fix', 'true')])

## ComPWA particle list

In [13]:
import pycompwa.ui

particles = pycompwa.ui.read_particles('../particle_list.xml')

2020-04-29 14:30:58,035 [INFO]  Particle a0(980)- with identical ID 9000211 already exists in list with the name a0(980)+ and ID 9000211. Particle properties will be overwritten!


2020-04-29 14:30:58,036 [INFO]  Particle PatricParticle with identical ID 1234 already exists in list with the name Chic1 and ID 1234. Particle properties will be overwritten!


2020-04-29 14:30:58,037 [INFO]  Particle nbar with identical ID 2112 already exists in list with the name n and ID 2112. Particle properties will be overwritten!


You can append particles to this instance from another list with

In [14]:
pycompwa.ui.Logging('error')  # switch off overwrite warnings
pycompwa.ui.insert_particles(particles, '../particle_list.xml')

2020-04-29 14:30:58,045 [INFO] Logging to file disabled!
